# Plotting Foliumn Maps
This notebook demontrates how the 'plot_folium' script works in practice.

In [2]:
import pandas as pd
from plot_folium import get_folium_map

In [7]:
help(get_folium_map)

Help on function get_folium_map in module plot_folium:

get_folium_map(df, latitude_col, longitude_col, colour_by=None, popup_name_col=None)
    Plots an interactive map of longitude latitude coordinates in Folium.
    
    df:             A dataframe with a longitude and latitude coordinates within.
    latitude_col    The column name containing latitude figures
    longitude_col   The column name containing longitude figures
    colour_by       Any column name containing the categories you wish to colour by
    pop_name_col    Enter the column name which contains any popup information you wish to display
                    when you click  point



In [16]:
# Looad in data. Below is a dataset from Crunchbase showing all leading 
# startups from Nottingham.

df = pd.read_csv('test_data.csv')
df.sample(3)

,iid,organization_name,description,cb_rank_(company),categories,headquarters_location,city,number_of_employees,coord,latitude,...,founded_date,founded_date_precision,website,funding_status,last_funding_type,last_funding_amount_currency_(in_usd),similarweb__global_traffic_rank,similarweb__average_visits_(6_months),ipo_status,colour
25,64680,Haemostatix,Haemostatix has developed a new class of activ...,69819.0,"Clinical Trials, Health Care, Medical","Nottingham, Nottingham, United Kingdom",Nottingham,NaN,"[52.9508143, -1.1364618]",52.950814,...,2003-01-01,year,http://www.haemostatix.com/home.html,M&A,Venture - Series Unknown,98082.0,NaN,97.0,Private,test_category_1
208,64921,iLocked,Ilocked is an escape room company located in N...,545066.0,Local Business,"Nottingham, Nottingham, United Kingdom",Nottingham,1-10,"[52.95550069999999, -1.1419327]",52.955501,...,NaN,NaN,https://iamlocked.co.uk/,NaN,NaN,NaN,5777760.0,7137.0,Private,test_category_3
96,64762,Nottingham Building Society,Nottingham Building Society is a company that ...,223880.0,"Banking, Financial Services","Nottingham, Nottingham, United Kingdom",Nottingham,501-1000,"[52.95501290000001, -1.1483992]",52.955013,...,NaN,NaN,https://www.thenottingham.com,NaN,NaN,NaN,522222.0,49836.0,Private,test_category_1


In [14]:
df = m = get_folium_map(df, 'latitude', 'longitude', colour_by = 'colour', popup_name_col = 'organization_name')

In [138]:
def get_folium_map(df, latitude_col,longitude_col, colour_by = None, popup_name_col = None):
  '''
  Plots an interactive map of longitude latitude coordinates in Folium.
  
  df:             A dataframe with a longitude and latitude coordinates within.
  latitude_col    The column name containing latitude figures
  longitude_col   The column name containing longitude figures
  colour_by       Any column name containing the categories you wish to colour by
  pop_name_col    Enter the column name which contains any popup information you wish to display
                  when you click  point
  '''

  import pandas as pd
  import folium
  import numpy as np
  
  global colourdict
  
  # Set up a list of points
  locationlist = df[[latitude_col, longitude_col]].values.tolist()


  # Find the mean long/lat coordinates so we an use these as the maps centre point
  mean_lat = df[latitude_col].mean()
  mean_long = df[longitude_col].mean()

  m = folium.Map(location=[mean_lat, mean_long], 
                     #zoom_start=zoom, 
                     prefer_canvas=True,
                     zoom_control= True)


  # Check if a colour column has been defined
  # if so, set up a colour dictionary to define some colours
  if colour_by:    
    colour_options = ['blue', 'red', 'green', 'darkgreen', 'darkpurple', 'darkred', 'gray', 'darkblue', 'lightblue', 'lightgray', 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'cadetblue','black']
    categories = np.unique(df[colour_by])
    colours = np.linspace(0, 1, len(categories))
    colourdict = dict(zip(categories, colours))

    colour_no = 0 # counter for keep track of max colours

    for i in colourdict.keys():
      colourdict[i] = colour_options[colour_no]
      colour_no += 1
      if colour_no == len(colour_options): colour_no = 0 #reset counter if we reach the max



  for point in range(0, len(locationlist)):

      # Check if a popup name column has been defined
      if popup_name_col:
        popup = folium.Popup(df.iloc[point][popup_name_col], parse_html=True)
      else:
        popup = None


      # Check if a colour column has been defined
      if colour_by:
        colour = colourdict[df.iloc[point][colour_by]]
      else:
        colour = 'red'


      folium.CircleMarker(locationlist[point], 
                          radius=1, 
                          color=colour,
                          popup=popup
                         ).add_to(m)

  m

  return m